In [1]:
import math
from random import random

class vec3 :
    def __init__(self, x, y, z) :
        self.x = x
        self.y = y
        self.z = z
        
    def length(self) :
        return math.sqrt( self.x * self.x + self.y * self.y + self.z * self.z )
    
    def normalize(self) :
        v_length = self.length()
        self.x = self.x / v_length
        self.y = self.y / v_length
        self.z = self.z / v_length
    
    def dot(self, other_v) :
        return self.x * other_v.x + self.y * other_v.y + self.z * other_v.z
    
    def __add__(self, other_v) :
        if isinstance(other_v, self.__class__):
            return vec3( self.x + other_v.x, self.y + other_v.y, self.z + other_v.z )
        else :
            return vec3( self.x + other_v, self.y + other_v, self.z + other_v )
    
    def __sub__(self, other_v) :
        if isinstance(other_v, self.__class__):
            return vec3( self.x - other_v.x, self.y - other_v.y, self.z - other_v.z )
        else :
            return vec3( self.x - other_v, self.y - other_v, self.z - other_v )
    
    def __mul__(self, other_v) :
        if isinstance(other_v, self.__class__):
            return vec3( self.x * other_v.x, self.y * other_v.y, self.z * other_v.z )
        else :
            return vec3( self.x * other_v, self.y * other_v, self.z * other_v )
    
    def __truediv__(self, other_v) :
        if isinstance(other_v, self.__class__):
            return vec3( self.x / other_v.x, self.y / other_v.y, self.z / other_v.z )
        else :
            return vec3( self.x / other_v, self.y / other_v, self.z / other_v )

In [2]:
class ray :
    def __init__(self, v1, v2) :
        self.A = v1
        self.B = v2
        
    def point_at(self, t) :
        return self.A + self.B * t
        
    def get_direction(self) :
        return self.B
        
    def get_origin(self) :
        return self.A

In [3]:
class hit_record : 
    def __init__(self) :
        self.hit = False
        self.t = 1000000.0
        self.normal = vec3(0.0, 0.0, 0.0)
        self.p = vec3(0.0, 0.0, 0.0)

class sphere :
    def __init__(self, center, radius) :
        self.center = center
        self.radius = radius
                
    def hit_sphere(self, r, record) :
        oc = r.get_origin() - self.center
        direction = r.get_direction()
        a = direction.dot( direction )
        b = 2.0 * oc.dot(direction)
        c = oc.dot(oc) - self.radius * self.radius
        D = b * b - 4* a * c
        
        if D > 0.0 :
            temp_t = (-b - math.sqrt(D)) / (2.0 * a)
            if temp_t > 0.0 and temp_t < record.t :
                record.t = temp_t
                record.p = r.point_at(temp_t)
                record.normal = record.p - self.center
                record.normal.normalize()
                record.hit = True
            temp_t = (-b + math.sqrt(D)) / (2.0 * a)
            if temp_t > 0.01 and temp_t < record.t :
                record.t = temp_t
                record.p = r.point_at(temp_t)
                record.normal = record.p - self.center
                record.normal.normalize()
                record.hit = True
        
        return record

In [4]:
class camera :
    def __init__(self, origin, left_bottom, horizontal, vertical) :
        self.origin = origin
        self.left_bottom = left_bottom
        self.horizontal = horizontal
        self.vertical = vertical
        
    def get_ray(self, u, v) :
        target_pos = self.left_bottom + self.horizontal * u + self.vertical * v
        target_dir = target_pos - self.origin
        target_dir.normalize()
        return ray(self.origin, target_dir)

In [5]:
def get_random_in_unit_sphere() :
    p = vec3(random(), random(), random()) * 2.0 - vec3(1.0, 1.0, 1.0)
    while p.length() ** 2 >= 1 :
        p = vec3(random(), random(), random()) * 2.0 - vec3(1.0, 1.0, 1.0)
    return p    

In [6]:
def color(r, s_list) :
    record = hit_record()
    for s in s_list :
        record = s.hit_sphere(r, record)
    
    if record.hit == True :
        
        return color(ray( record.p, new_dir ), s_list) * 0.5
     
    dir = r.get_direction()
    dir.normalize()
    t = (dir.y + 1.0) * 0.5
    return vec3(0.5, 0.7, 1.0) * t + vec3(1.0, 1.0, 1.0) * (1.0 - t) 

In [7]:
width = 800
height = 400
GAMMA = 2.2
num_samples = 60

origin_pos      = vec3(0.0 , 0.0 , 0.0 )
left_bottom_pos = vec3(-2.0, -1.0, -1.0)
horizontal      = vec3(4.0 , 0.0 , 0.0 )
vertical        = vec3(0.0 , 2.0 , 0.0 )

cam = camera(origin_pos, left_bottom_pos, horizontal, vertical)

s_list = [ 
    sphere(vec3(0.0,    0.0, -1.0),   0.5), 
    sphere(vec3(0.0, -100.5, -1.0), 100.0)
]

with open("test60_gamma_corrected3.ppm", 'w') as img :
    img.write("P3\n{0:d} {1:d}\n255\n".format(width, height))
    for j in range(height - 1, -1, -1) :
        for i in range(width) :
            col = vec3(0.0, 0.0, 0.0)
            for s in range(num_samples) :
                u = float(i + random()) / width
                v = float(j + random()) / height
                cam_ray = cam.get_ray(u, v)
                col = col + color(cam_ray, s_list)
            col = col / float(num_samples)
            
            col.x = col.x ** (1.0 / GAMMA)
            col.y = col.y ** (1.0 / GAMMA)
            col.z = col.z ** (1.0 / GAMMA)
            
            ir = int(col.x * 255)
            ig = int(col.y * 255)
            ib = int(col.z * 255)
            
            img.write("{0:d} {1:d} {2:d}\n".format(ir, ig, ib))

NameError: name 'new_dir' is not defined